In [11]:
from transformers import AutoTokenizer               # <-- You forgot this import

from evaluation.evaluator import EarlyExitEvaluator
from strategies.confidence_exit import ConfidenceExit
from models.gpt2_wrapper import GPT2WithEarlyExit
from evaluation.dataset_loaders.sst2 import load_sst2

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [6]:
strategy = ConfidenceExit(threshold=0.8, allowed_layers=[3,6,9])
model = GPT2WithEarlyExit("gpt2", strategy, tokenizer)

In [7]:
dataset = load_sst2(fraction=0.10)   # <-- use 10% data

evaluator = EarlyExitEvaluator(tokenizer)

result = evaluator.evaluate(
    model=model,
    strategy=strategy,
    dataset=dataset,
    task_type="classification",
)

print(result)

Evaluating: 100%|███████████████████████████████| 87/87 [00:01<00:00, 54.04it/s]

{'metric': 'accuracy', 'score': np.float64(0.0), 'avg_latency_sec': np.float64(0.018404911304342336), 'tokens_per_sec': 54.33332350610494, 'avg_layers_used': np.float64(5.114942528735632), 'num_samples': 87}


In [9]:
from evaluation.dataset_loaders.agnews import load_agnews
from evaluation.dataset_loaders.cnn_dm import load_cnndm
from evaluation.dataset_loaders.squad import load_squad
from evaluation.dataset_loaders.wmt_en_fr import load_wmt_enfr

datasets = [
    ("sst2", load_sst2, "classification"),
    ("agnews", load_agnews, "classification"),
    ("cnn_dm", load_cnndm, "summarization"),
    ("wmt14_enfr", load_wmt_enfr, "translation"),
    ("squad", load_squad, "qa"),
]

for name, loader, task in datasets:
    print(f"Testing {name}...")

    dataset = loader(fraction=0.10)

    result = evaluator.evaluate(
        model=model,
        strategy=strategy,
        dataset=dataset,
        task_type=task,
    )

    print(name, result)

Testing sst2...


Evaluating: 100%|███████████████████████████████| 87/87 [00:01<00:00, 53.20it/s]


sst2 {'metric': 'accuracy', 'score': np.float64(0.0), 'avg_latency_sec': np.float64(0.018698522414284192), 'tokens_per_sec': 53.48016157876085, 'avg_layers_used': np.float64(5.114942528735632), 'num_samples': 87}
Testing agnews...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Map:   0%|          | 0/1057 [00:00<?, ? examples/s]

Evaluating: 100%|███████████████████████████| 1057/1057 [00:51<00:00, 20.64it/s]

squad {'metric': 'token_f1', 'score': np.float64(0.016823466137562637), 'avg_latency_sec': np.float64(0.04826024555325396), 'tokens_per_sec': 20.72098864264015, 'avg_layers_used': np.float64(6.811731315042573), 'num_samples': 1057}


In [ ]:
### Strategy 2 - Confidence threshold should be (meet) in Continous layers

In [12]:
from strategies.continous_confidence_exit import ContinuousConfidenceExit

strategy = ContinuousConfidenceExit(
    threshold=0.75,
    required_consecutive=2,
    allowed_layers=[3, 6, 9, 11]
)

model = GPT2WithEarlyExit("gpt2", strategy, tokenizer)
evaluator = EarlyExitEvaluator(tokenizer)

In [ ]:
datasets = [
    ("sst2", load_sst2, "classification"),
    ("agnews", load_agnews, "classification"),
    ("cnn_dm", load_cnndm, "summarization"),
    ("wmt14_enfr", load_wmt_enfr, "translation"),
    ("squad", load_squad, "qa"),
]

for name, loader, task in datasets:
    print(f"\n========== Testing {name.upper()} ==========\n")

    # Use 2% of dataset
    dataset = loader(fraction=0.02)

    result = evaluator.evaluate(
        model=model,
        strategy=strategy,
        dataset=dataset,
        task_type=task,
    )

    print(name, result)


========== Testing SST2 ==========



Map:   0%|          | 0/17 [00:00<?, ? examples/s]

Evaluating: 100%|███████████████████████████████| 17/17 [00:00<00:00, 21.62it/s]


sst2 {'metric': 'accuracy', 'score': np.float64(0.0), 'avg_latency_sec': np.float64(0.04608050514669979), 'tokens_per_sec': 21.70115099251724, 'avg_layers_used': np.float64(12.0), 'num_samples': 17}

========== Testing AGNEWS ==========



Map:   0%|          | 0/152 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/267 [00:00<?, ? examples/s]

Evaluating: 100%|█████████████████████████████| 267/267 [01:19<00:00,  3.34it/s]


cnn_dm {'metric': 'rougeL', 'score': np.float64(0.029517060847254528), 'avg_latency_sec': np.float64(0.2987214992108863), 'tokens_per_sec': 3.3475996961773316, 'avg_layers_used': np.float64(12.0), 'num_samples': 267}

========== Testing WMT14_ENFR ==========



Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/30 [00:00<?, ?it/s]